<a href="https://colab.research.google.com/github/suplab/amazon-bedrock-genai-labs/blob/main/03_ConverseAPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Converse APIs

The Amazon Bedrock Converse API enables developers to build sophisticated conversational applications while managing context across multiple interactions. Unlike standard foundation model calls which are stateless, the Converse API provides a structured way to maintain conversation history and context, making it ideal for chatbots, virtual assistants, and other interactive applications.

Converse API also provides a unified way to invoke foundation models, making it easier to swap out models when needed.

## Building Blocks of the Converse API
Now that we understand what the Converse API is and its purpose, let's examine the essential components that make it work. Each component plays a specific role in creating effective conversational experiences, and understanding how they interact is crucial for building robust applications.

From system prompts that shape the model's behavior to tool configurations that enable real-world actions, these building blocks can be combined in various ways to create exactly the conversational experience you need.

Let's break down each component, explore practical examples, and review best practices for implementation.

### System prompt
A system prompt defines the model's behavior, personality, and operating parameters. It sets the foundation for how the model should interact and what type of responses it should provide.

```
system_prompt = "You are an AI assistant that helps create playlists. Only return song names and artists."
```

### Message Array
Message arrays store the conversation history as a sequence of interactions between the user and the assistant. Each message includes a role identifier and the content of the message.

```
messages = [
    {
        "role": "user",
        "content": "Create a playlist of 3 pop songs"
    }
]
```

### Inference Parameters
Inference parameters control how the model generates responses. Different models support different parameters, so it's important to check the model-specific documentation.Example for Claude models:

```
inference_config = {
    "temperature": 0.7,
    "max_tokens": 1024,
    "top_p": 0.999,
}
additional_model_fields = {"top_k": top_k}
```

### Invoking the Converse API

```
response = bedrock.converse(
    modelId="<MODEL_ID_CLAUDE>",
    messages=messages,
    system=system_prompt,
    inferenceConfig=inference_config,
    additionalModelRequestFields=additional_model_fields
)
```

### Tool Configuration
Tool configurations enable the model to interact with external functions.Here's an example of defining a weather tool and then passing the tool to the Converse API:

```
weather_tool = {
    "toolSpec": {
        "name": "getWeather",
        "description": "Gets the current weather using latitude and longitude.",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "lat": {"type": "number"},
                    "lon": {"type": "number"}
                },
                "required": ["lat", "lon"]
            }
        }
    }
}

initial_response = bedrock.converse(
    modelId="<MODEL_ID>",
    system="<system_prompt>",
    messages=["<initial_user_msg>"],
    toolConfig={
        "tools": [weather_tool],
        "toolChoice": {"auto": {}}
    },
    inferenceConfig={"temperature": 0.7}
)
```

## Setting up Bedrock

In [ ]:
!pip install boto3

In [ ]:
import boto3
import json

bedrock = boto3.client('bedrock−runtime', region_name='us-east-1')
bedrock_agent = boto3.client(service_name='bedrock−agent', region_name='us-east-1')

MODEL_ID = "amazon.nova−micro−v1:0"

## Traveling with Bedrock

In [ ]:
temperature = .7

inference_config = {"temperature": temperature}

system_prompts = [{"text": "You are a virtual travel assistant that suggests destinations based on user preferences."
                + "Only return destination names and a brief description."}]

messages = []

message_1 = {
    "role": "user",
    "content": [{"text": "Create a list of 3 travel destinations."}]
}

messages.append(message_1)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config
)

def print_response(response):
    model_response = response.get('output', {}).get('message', {}).get('content', [{}])[0].get('text', '')

    print("✈️ Your suggested travel destinations:")
    print(model_response)

print_response(response)

## Continuing the conversation

In [ ]:
message_2 = {
        "role": "user",
        "content": [{"text": "Only suggest travel locations that are no more than one short flight away."}]
}

messages.append(message_2)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config
)

print_response(response)

## Refine the results

In [ ]:
message_3 = {
        "role": "user",
        "content": [{"text": "List destinations by their proximity to Seattle, WA USA"}]
}

messages.append(message_3)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config
)

print_response(response)

## Creating a Bedrock Prompt

In [ ]:
try:
    response = bedrock_agent.create_prompt(
        name="Travel-Agent-Prompt",
        description="Checks if all trip information has been provided.",
        variants=[
            {
                "name": "Variant1",
                "modelId": MODEL_ID,
                "templateType": "CHAT",
                "inferenceConfiguration": {
                    "text": {
                        "temperature": 0.4
                    }
                },
                "templateConfiguration": {
                    "chat": {
                        'system': [
                            {
                                "text": """You are a travel agent evaluating trip requests for custom itineraries.
                                Review the message carefully and answer YES or NO to the following screening questions.
                                Be strict—if any detail is missing or unclear, answer NO.

                                A) Is the destination clearly stated?
                                B) Are the travel dates within a reasonable range (not last−minute or over a year away)?
                                C) Does the request avoid high−risk or restricted activities (e.g., extreme sports, off−grid travel)?
                                D) Is there any mention of a valid passport or travel documentation?
                                E) Is there enough information to follow up with a proposed itinerary?"""
                            }
                        ],
                        'messages': [{
                            'role': 'user',
                            'content': [
                                {
                                    'text': "Trip request: {{event_request}}"
                                }
                            ]
                        }],
                        'inputVariables' : [
                            { 'name' : 'event_request'}
                        ]
                    }
                }
        }]
    )
    print("Created!")
    prompt_arn = response.get("arn")
except bedrock.exceptions.ConflictException as e:
    print("Already exists!")
    response = bedrock.list_prompts()
    prompt = next((prompt for prompt in response['promptSummaries'] if prompt['name'] == "TripBooker_xyz"), None)
    prompt_arn = prompt['arn']

prompt_arn

The output should look similar to this: `Created! arn:aws:bedrock:us-east-1:12345678910:prompt/5E0L1VBVM1`

In [ ]:
response = bedrock.converse(
    modelId=prompt_arn,
    promptVariables={
        'event_request': {
            'text': """
                Hi there! I'm planning a trip to Italy with my partner and would love some help organizing the itinerary. We're hoping to travel between September 10–20 this year, ideally flying into Rome and spending a few days in Florence and Venice as well. We’d love recommendations on tours, cultural sites, and good local restaurants. We’re not interested in anything risky like skydiving or hiking remote trails — just want a relaxing and enriching experience. We both have valid passports. Let me know what other details you need!
                """
        }
    },
)
print(response['output']['message']['content'][0]['text'])

## Creating an AWS Lambda function

Bedrock and Lambda are two tools that you can pair together that compliment each other well. While LLMs are very powerful, they are unable to do everything you need them to do. Bedrock is able to reach out to Lambda to call functions on your behalf. This allows you to "build" in new features to Bedrock.

In this task, you'll be creating a Lambda function that performs mathematical operations. In a future task, this will be utilized by Bedrock.



In [ ]:
import json

def add(num1, num2):
    return num1 + num2

def subtract(num1, num2):
    return num1 − num2

def multiply(num1, num2):
    return num1 * num2

def divide(num1, num2):
    if num2 == 0:
        raise ValueError("division by zero")
    return num1 / num2

def power(num1, num2):
    return num1 ** num2

def lambda_handler(event, context):
    operation = event.get('operation')
    try:
        num1 = float(event['num1'])
        num2 = float(event['num2'])
    except (KeyError, ValueError):
        return {
            'statusCode': 400,
            'body': json.dumps({'error': 'Invalid input, num1 and num2 must be numeric'})
        }

    # Map operations to functions
    operations = {
        'add': add,
        'subtract': subtract,
        'multiply': multiply,
        'divide': divide,
        'power': power
    }

    # Find the operation function and execute
    operation_func = operations.get(operation)

    if operation_func:
        try:
            result = operation_func(num1, num2)
            return {
                'statusCode': 200,
                'body': json.dumps({'result': result})
            }
        except ValueError as e:
            return {
                'statusCode': 400,
                'body': json.dumps({'error': str(e)})
            }
    else:
        return {
            'statusCode': 400,
            'body': json.dumps({'error': 'invalid operation'})
        }

## Integrating Bedrock and Lambda

In [ ]:
# Setup Bedrock and Lambda
import boto3
import json

bedrock = boto3.client(service_name='bedrock-runtime')
lambda_client = boto3.client("lambda")
MODEL_ID = "amazon.nova-micro-v1:0"

# Define the calculation tool
math_tool = {
    "toolSpec": {
        "name": "calculateNumbers",
        "description": "Performs basic arithmetic operations",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "operation": {"type": "string"},
                    "num1": {"type": "number"},
                    "num2": {"type": "number"}
                },
                "required": ["operation", "num1", "num2"]
            }
        }
    }
}

# Function to trigger the Lambda calculation service
def execute_calculation(input_data):
    response = lambda_client.invoke(
        FunctionName="math-function",
        InvocationType="RequestResponse",
        Payload=json.dumps(input_data)
    )
    response_payload = response["Payload"].read()
    calculation_result = json.loads(response_payload)
    response_body = calculation_result.get("body", "{}")
    return json.loads(response_body) if isinstance(response_body, str) else response_body

# User's initial message
user_input = {
    "role": "user",
    "content": [{"text": "Please subtract 60 from 100"}]
}

# Define system instructions
system_instructions = [
    {"text": """
    You are a virtual assistant capable of performing basic arithmetic operations: add, subtract, multiply, and divide.
    If the user doesn't specify an operation, ask them for more details.
    """}
]

# First interaction with the model
first_interaction = bedrock.converse(
    modelId=MODEL_ID,
    system=system_instructions,
    messages=[user_input],
    toolConfig={
        "tools": [math_tool],
        "toolChoice": {"auto": {}}
    },
    inferenceConfig={"temperature": 0.7}
)

# Process the assistant's response to check if tool is required
assistant_reply = first_interaction["output"]["message"]
message_parts = assistant_reply["content"]
tool_request_block = next((part for part in message_parts if "toolUse" in part), None)

if not tool_request_block:
    print("=== Assistant's Direct Response ===")
    print(message_parts[0]["text"])
else:
    tool_request = tool_request_block["toolUse"]
    tool_input_data = tool_request["input"]
    tool_id = tool_request["toolUseId"]
    print(tool_request_block)
    print(f"→ Assistant triggered tool: calculateNumbers with input: {tool_input_data}")

    # Execute the requested tool
    tool_result = execute_calculation(tool_input_data)
    print(f"← Lambda Function output: {tool_result}")

    # Create a response based on the tool's output
    try:
        result_summary = f"The outcome of the calculation is {tool_result['result']}."
    except Exception as e:
        result_summary = f"Oops! There was an error with the calculation. ({str(e)})"

    # Generate tool result message
    tool_response_msg = {
        "role": "user",
        "content": [
            {
                "toolResult": {
                    "toolUseId": tool_id,
                    "content": [{"text": result_summary}]
                }
            }
        ]
    }

    # Send tool result back to the model
    final_output = bedrock.converse(
        modelId=MODEL_ID,
        messages=[user_input, assistant_reply, tool_response_msg],
        toolConfig={
            "tools": [math_tool],
            "toolChoice": {"auto": {}}
        },
        inferenceConfig={"temperature": 0.7}
    )

    # Display the final response from the assistant
    final_message = final_output["output"]["message"]["content"][0]["text"]
    print("\n=== Final Assistant Response ===")
    print(final_message)

Your output should look similar to this example:

```
{'toolUse': {'toolUseId': 'tooluse_mCTQPfNcTvGmiclwGbD2bA', 'name': 'calculateNumbers', 'input': {'num1': 100, 'operation': 'subtract', 'num2': 60}}}
Assistant triggered tool: calculateNumbers with input: {'num1': 100, 'operation': 'subtract', 'num2': 60}
Lambda Function output: {'result': 40.0}

=== Final Assistant Response ===
The outcome of subtracting 60 from 100 is 40.0.
```